In [34]:
import eigen as e
import rbdyn as rbd
import sva as s
from rbdyn.parsers import *

import numpy as np

urdf_file = "./assets/kuka_iiwa.urdf"

robot = from_urdf_file(urdf_file)
robot.mbc.gravity = e.Vector3d(0, 0, 9.81)
robot.mbc.zero(robot.mb)

q = np.array([1,2,4,4,5,6,3])
dq = -np.array([1,2,3,4,5,6,7])

robot.mbc.q = [
    [],
    [q[0]],
    [q[1]],
    [q[2]],
    [q[3]],
    [q[4]],
    [q[5]],
    [q[6]],
]
robot.mbc.alpha = [
    [],
    [dq[0]],
    [dq[1]],
    [dq[2]],
    [dq[3]],
    [dq[4]],
    [dq[5]],
    [dq[6]],
]

sel_mat = e.MatrixXd.Zero(6, 6)
sel_mat.coeff(0, 3, 1)
sel_mat.coeff(1, 4, 1)
sel_mat.coeff(2, 5, 1)
sel_mat.coeff(3, 0, 1)
sel_mat.coeff(4, 1, 1)
sel_mat.coeff(5, 2, 1)
        
rbd.forwardKinematics(robot.mb, robot.mbc)
rbd.forwardVelocity(robot.mb, robot.mbc)

fd = rbd.ForwardDynamics(robot.mb)
fd.computeH(robot.mb, robot.mbc)
M = fd.H()
Minv = M.inverse()
fd.computeC(robot.mb, robot.mbc)
h = fd.C()

jac = rbd.Jacobian(robot.mb, robot.mb.bodies()[-1].name())
swapped_jac = e.MatrixXd(6, robot.mb.nrDof())
jac.fullJacobian(robot.mb, jac.jacobian(robot.mb, robot.mbc), swapped_jac)
J = sel_mat * swapped_jac

Lambda_inv = J*Minv*J.transpose()
mat_np = np.array(Lambda_inv)
reg = 0 * np.identity(np.size(mat_np,0))
Lambda = Lambda_inv.inverse()


In [37]:
print(jac.jacobian(robot.mb, robot.mbc))

print(Lambda)

           0    -0.841471     0.491295    -0.379859     0.272047    -0.740069     0.433066
           0     0.540302     0.765147     0.618179    -0.636367    -0.617803    -0.740084
           1  4.89659e-12    -0.416147     0.688159     0.721822     -0.26574     0.514518
   -0.066815    0.0615659     0.114991     0.353655 -3.41422e-17            0            0
    0.315163    0.0958833    -0.187136     0.184561  2.56409e-17           -0            0
           0    -0.226506     -0.20832    0.0294225  3.54731e-17            0            0
   4.46314    -1.4827    1.94288  0.0982833 -0.0471102  -0.150488
   -1.4827    7.96699    -4.8593   0.142771  -0.075609  -0.228926
   1.94288    -4.8593     8.6724 -0.0727542  0.0317979   0.106975
 0.0982833   0.142771 -0.0727542  0.0349813 -0.0139804 -0.0487113
-0.0471102  -0.075609  0.0317979 -0.0139804  0.0111976  0.0264355
 -0.150488  -0.228926   0.106975 -0.0487113  0.0264355  0.0800248


In [30]:
%matplotlib widget
import ipywidgets as widgets
import os
import matplotlib.pyplot as plt
from pytransform3d.urdf import UrdfTransformManager
import math

plt.rcParams['figure.dpi'] = 150

BASE_DIR = "assets/"
data_dir = BASE_DIR
search_path = "."
while (not os.path.exists(data_dir) and
       os.path.dirname(search_path) != "riemannian_mot_pol"):
    search_path = os.path.join(search_path, "..")
    data_dir = os.path.join(search_path, BASE_DIR)

tm = UrdfTransformManager()
with open(os.path.join(data_dir, "kuka_iiwa.urdf"), "r") as f:
    tm.load_urdf(f.read(), mesh_path=data_dir)

ax = tm.plot_frames_in(
    "lbr_iiwa_link_0", s=0.2, whitelist=["lbr_iiwa_link_0", "lbr_iiwa_link_1", "lbr_iiwa_link_7"],
    show_name=True) # plot all frames in a given reference frame
ax = tm.plot_connections_in("lbr_iiwa_link_0", ax=ax) # plot connections between parent and child frames
tm.plot_visuals("lbr_iiwa_link_0", ax=ax, wireframe=False, convex_hull_of_mesh=False) # plot all visuals in a given reference frame

ax.set_xlim((-1, 1))
ax.set_ylim((-1, 1))
ax.set_zlim((0.0, 1))
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
%matplotlib widget
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
from pytransform3d.transformations import plot_transform
from pytransform3d.plot_utils import plot_sphere, remove_frame


random_state = np.random.RandomState(42)
ax = plot_sphere(
    radius=0.5, wireframe=False, alpha=0.1, color="k", n_steps=20, ax_s=1)
plot_sphere(ax=ax, radius=0.5, wireframe=True)
plot_transform(ax=ax, A2B=np.eye(4), s=0.3, lw=3)
remove_frame(ax)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
%matplotlib widget
import matplotlib.pyplot as plt
from pytransform3d.transformations import plot_transform
from pytransform3d.plot_utils import make_3d_axis

ax = make_3d_axis(ax_s=1, unit="m", n_ticks=6)
plot_transform(ax=ax)
plt.tight_layout()
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …